# Discovery of Frequent Itemsets and Association Rules

##### Group 16

The problem of discovering association rules between itemsets in a sales transaction database (a set of baskets) includes the following two sub-problems:

1. Finding frequent itemsets with support at least s;
2. Generating association rules with confidence at least c from the itemsets found in the first step.
Remind that an association rule is an implication X → Y, where X and Y are itemsets such that X∩Y=∅. Support of the rule X → Y is the number of transactions that contain X⋃Y. Confidence of the rule X → Y the fraction of transactions containing X⋃Y in all transactions that contain X.

#### Exercise 1
You are to solve the first sub-problem: to implement the A-Priori algorithm for finding frequent itemsets with support at least s in a dataset of sales transactions. Remind that support of an itemset is the number of transactions containing the itemset. To test and evaluate your implementation, write a program that uses your A-Priori algorithm implementation to discover frequent itemsets with support at least s in a given dataset of sales transactions which includes generated transactions (baskets) of hashed items (see Canvas).

In [1]:
baskets = [i.strip().split() for i in open("T10I4D100K.dat").readlines()]
#baskets = baskets[:5] # used as test subset

In [2]:
transactions = {} # Dictionary with transaction ID as key, and basket as value
count = 0
for basket in baskets:
    count += 1
    transactions[count] = basket

In [3]:
items = set() # Set of items from all baskets
for i in transactions.values():
    for j in i:
        items.add(j) 

In [4]:
# Count the frequency of each item
def freq(k,items, transactions):
    items_counts = dict() # Dictionary of item and its frequency
    for i in items:
        if k == 1:
            temp_i = {i}
        else:
            temp_i = set(i)
            
        for j in transactions.items(): # and basket
            if temp_i.issubset(set(j[1])): # if item is in basket
                if i in items_counts:
                    items_counts[i] += 1 # If already spotted/already in item-freq dict, add 1 to count
                else:
                    items_counts[i] = 1 # If not spotted yet, set count to 1
    return items_counts

We set s_min for itemsets of size 1 to 5000 which is relatively high, however it helps us run the functions in demonstration-friendly time. 

In [5]:
# Support
s_min = 5000
base = freq(1,items, transactions)
L1 = [{j[0]:j[1] for j in base.items() if j[1]>=s_min}]  #frequent items 

### Candidate generation function
As input, the user should pass the number of items k of the k-itemset, and the previous' size frequent itemsets.

In [7]:
from itertools import combinations

# Candidates of len-k which are generated by combining itemsets from L_k-1 and L_1
def C_k(k, prev_freq):
    cand = []
    if k-1 == 1:
        combs = combinations(list(L1[0].keys()), k)
        cand = list(combs)
        
    else:
        combs = combinations(list(L1[0].keys()), k)
        cand = list(combs)

        for i in prev_freq[0].keys():
                temp = set(i)
                for j in L1[0].keys():
                    if len(temp.union({j}))==k:
                        cand.append(tuple(temp.union({j})))
    # Remove duplicate tuples 
    cand = [t for t in (set(tuple(i) for i in cand))]
    return cand

#### Example of candidates function and its output (i guess the following will be deleted?)

## Get frequent items
L1 already found and used for getting the candidates C_k as a combination of L1 and L_(k-1). The default threshold for support is 10 in order to get some interesting results. The generation of candidates and eventually frequent itemsets stops when there are no new frequent itemsets discovered, otherwise it continues with the candidates and frequent itemsets of the next size. 

In [8]:
#Look for frequent items until there is no one
lookup = [] # acts as lookup dictionary later on incl frozensets so order doesn't matter
size = 1
frequent_items = []
s_min = 10
#base = freq(1,items, transactions)
#L1 = [{j[0]:j[1] for j in base.items() if j[1]>=1}]
lookup.append({frozenset([k]): v for k, v in base.items()})

for x in list(L1[0].keys()):
    frequent_items.append(tuple({x}))
prev_freq = L1
while True: 
    size+=1
    candidates = C_k(size,prev_freq)
    lvl = freq(size,candidates,transactions)
    frequents = [{j[0]:j[1] for j in lvl.items() if j[1]>=s_min}]
    prev_freq = frequents
    if len(frequents[0])!=0:
        frequent_items.extend(list(frequents[0].keys()))
        lookup.append({frozenset(k): v for k, v in lvl.items()})
    else:
        break


In [9]:
print(frequent_items)

[('722',), ('529',), ('354',), ('494',), ('217',), ('368',), ('829',), ('419',), ('766',), ('684',), ('217', '419'), ('829', '766'), ('722', '419'), ('829', '684'), ('368', '419'), ('217', '766'), ('494', '829'), ('722', '766'), ('368', '829'), ('722', '354'), ('217', '684'), ('494', '217'), ('217', '368'), ('529', '829'), ('722', '684'), ('368', '766'), ('722', '368'), ('494', '419'), ('529', '217'), ('529', '494'), ('368', '684'), ('354', '829'), ('529', '419'), ('354', '217'), ('354', '494'), ('494', '766'), ('494', '368'), ('529', '766'), ('354', '419'), ('494', '684'), ('529', '354'), ('722', '529'), ('529', '368'), ('529', '684'), ('354', '766'), ('354', '684'), ('419', '766'), ('354', '368'), ('217', '829'), ('829', '419'), ('722', '829'), ('419', '684'), ('766', '684'), ('722', '217'), ('722', '494'), ('684', '722', '494'), ('419', '368', '494'), ('217', '684', '529'), ('217', '529', '419'), ('722', '529', '354'), ('722', '217', '829'), ('354', '494', '368'), ('368', '722', '49

#### Exercise 2 
Optional task for extra bonus: Solve the second sub-problem, i.e., develop and implement an algorithm for generating association rules between frequent itemsets discovered by using the A-Priori algorithm in a dataset of sales transactions. The rules must have support at least s and confidence at least c, where s and c are given as input parameters.

In [10]:
#association rule for itemsets of size >=2
fr = []
for f in frequent_items:
    if len(f)>1:
        fr.append(f)     

## Generation of all possible combinations of frequent itemsets that we got from Q1
We exclude null subsets and the subset itself so that we can then take for each subset A-->I\A

In [19]:
from itertools import chain, combinations
from copy import deepcopy

# For every subset A of frequent itemset I, rule is A -> I\A
def association_rules(frequents):
    lhs_rhs = []

    for itemset in frequents: # all subsets of itemset
        r = chain.from_iterable(combinations(itemset, r) for r in range(len(itemset)+1))
        final_r = []
        
        for com in list(r):
            if len(com)!=0 and len(com)!=len(itemset):
                final_r.append(com)   #all subsets of all frequent itemsets
        
        for A in final_r:
            remaining = set(final_r)-{A}
            temp = deepcopy(remaining)
            for rem in remaining:
                for a in A:
                    if {a}.issubset(rem):
                        if rem in temp:
                            temp.remove(rem)  #so that i won't have eg a->a,b
                            
            for rhs in temp:
                if [A,rhs] not in lhs_rhs:
                    lhs_rhs.append([A,rhs]) #pairs lhs,rhs-->if not already present so that we won't take same
                                                            #association rule twice (set was ruining order)
    print(lhs_rhs)
    return lhs_rhs


    pass

In [20]:
rules = association_rules(fr)

[[('217',), ('419',)], [('419',), ('217',)], [('829',), ('766',)], [('766',), ('829',)], [('722',), ('419',)], [('419',), ('722',)], [('829',), ('684',)], [('684',), ('829',)], [('368',), ('419',)], [('419',), ('368',)], [('217',), ('766',)], [('766',), ('217',)], [('494',), ('829',)], [('829',), ('494',)], [('722',), ('766',)], [('766',), ('722',)], [('368',), ('829',)], [('829',), ('368',)], [('722',), ('354',)], [('354',), ('722',)], [('217',), ('684',)], [('684',), ('217',)], [('494',), ('217',)], [('217',), ('494',)], [('217',), ('368',)], [('368',), ('217',)], [('529',), ('829',)], [('829',), ('529',)], [('722',), ('684',)], [('684',), ('722',)], [('368',), ('766',)], [('766',), ('368',)], [('722',), ('368',)], [('368',), ('722',)], [('494',), ('419',)], [('419',), ('494',)], [('529',), ('217',)], [('217',), ('529',)], [('529',), ('494',)], [('494',), ('529',)], [('368',), ('684',)], [('684',), ('368',)], [('354',), ('829',)], [('829',), ('354',)], [('529',), ('419',)], [('419',)

In [70]:
# Calculate confidence for each rule
min_c = 0.1
min_support = 7500

total_dict = {k: v for d in lookup for k, v in d.items()} # Create one dictionary as look-up
confidences = {}
supports = {}
for rule in rules:
    lhs = rule[0]
    rhs = rule[1]
    if frozenset(lhs) in total_dict:
        support = total_dict[frozenset(lhs)]
    union_lhs_rhs = frozenset(tuple(set(lhs+rhs)))
    if union_lhs_rhs in total_dict:
        support_union = total_dict[union_lhs_rhs]
        confidence = support_union/support
        confidences[(lhs,rhs)] = round(confidence,5)
        supports[(lhs,rhs)] = round(support,5)
association_rules_at_least_c = {j[0]:j[1] for j in confidences.items() if j[1]>=min_c}
association_rules_at_least_s = {j[0]:j[1] for j in supports.items() if j[1]>=min_support}
association_rules_at_least_c_s = association_rules_at_least_c.keys() & association_rules_at_least_s.keys()

In [71]:
association_rules_at_least_c

{(('368',), ('494',)): 0.10986,
 (('494',), ('368',)): 0.16856,
 (('829',), ('368',)): 0.17533,
 (('368',), ('829',)): 0.15253,
 (('684',), ('766',)): 0.11335,
 (('684', '419'), ('722',)): 0.12258,
 (('368', '766'), ('829',)): 0.13492,
 (('829', '766'), ('368',)): 0.21184,
 (('368', '722'), ('829',)): 0.11735,
 (('722', '829'), ('368',)): 0.15646,
 (('684', '494'), ('722',)): 0.10096,
 (('684', '829'), ('368',)): 0.33238,
 (('684', '368'), ('829',)): 0.29974,
 (('684', '529'), ('766',)): 0.11976,
 (('529', '766'), ('684',)): 0.12618,
 (('829', '722'), ('368',)): 0.15646,
 (('829', '419'), ('368',)): 0.16988,
 (('368', '419'), ('829',)): 0.12394,
 (('368', '354'), ('829',)): 0.14734,
 (('354', '829'), ('368',)): 0.18147,
 (('368', '494'), ('829',)): 0.10465,
 (('494', '829'), ('368',)): 0.33708,
 (('684', '494'), ('368',)): 0.16827,
 (('368', '217'), ('494',)): 0.11881,
 (('494', '217'), ('368',)): 0.19672,
 (('829', '494'), ('368',)): 0.33708,
 (('494', '684'), ('722',)): 0.10096,
 (('

In [66]:
print(total_dict[frozenset(('684', '829'))]) # Support of lhs
print(total_dict[frozenset(('684', '829', '368'))]) # Support of lhs+rhs
116/349

349
116


0.332378223495702

In [72]:
association_rules_at_least_s

{(('368',), ('419',)): 7828,
 (('368',), ('684',)): 7828,
 (('368',), ('494',)): 7828,
 (('368',), ('529',)): 7828,
 (('368',), ('217',)): 7828,
 (('368',), ('354',)): 7828,
 (('368',), ('766',)): 7828,
 (('368',), ('829',)): 7828,
 (('368',), ('722',)): 7828,
 (('368',), ('354', '529')): 7828,
 (('368',), ('217', '419')): 7828,
 (('368',), ('829', '766')): 7828,
 (('368',), ('684', '722')): 7828,
 (('368',), ('354', '766')): 7828,
 (('368',), ('529', '722')): 7828,
 (('368',), ('529', '419')): 7828,
 (('368',), ('722', '829')): 7828,
 (('368',), ('722', '766')): 7828,
 (('368',), ('684', '829')): 7828,
 (('368',), ('766', '217')): 7828,
 (('368',), ('529', '217')): 7828,
 (('368',), ('829', '722')): 7828,
 (('368',), ('829', '419')): 7828,
 (('368',), ('354', '829')): 7828,
 (('368',), ('419', '722')): 7828,
 (('368',), ('494', '829')): 7828,
 (('368',), ('684', '494')): 7828,
 (('368',), ('494', '217')): 7828,
 (('368',), ('829', '494')): 7828,
 (('368',), ('494', '766')): 7828,
 (('

In [73]:
association_rules_at_least_c_s

{(('368',), ('494',)), (('368',), ('829',))}